In [ ]:
## Pip install libraries (not included from anaconda)
########### 
%pip install google-play-scraper

#### TASK 1: GATHERING REVIEWS
Please note that the scraper code shouldn't be submitted, thus I have commented out the portion of code that does the scraping below

In [ ]:
# Importing libraries
import pandas as pd
import nltk
import string
import re
import time

# Importing outside library
######### Scraper libraries
from google_play_scraper import app #scrapes googleplay store
from google_play_scraper import Sort, reviews # Used to sort and fetch reviews from scraper
######################


nltk.download('punkt') # Need this on my computer (some computers don't), used to use nltk word_tokenize etc.
nltk.download('stopwords') # Need this on my computer (some computers don't), used to use nltk stopwords('english')
nltk.download('wordnet') # Need this on my computer (some computers don't), used to use nltk lemmatization 



# NOTE THIS WILL NOT BE RUNNED DUE TO TASK1 STATING, SCRAPPER CODE WILL NOT BE SUBMITTED
# This method gets reviews based on given app packagename, first param gets scraped
# Second param packageName is used for the name of the csv file
def get_reviews(scrapePackageNames, originalPackageName):

    appReviews = pd.DataFrame(columns=['Package name', 'Reviewer name', 'Review', 'Rating']) # initalize dataframe

    for scrapePackageName in scrapePackageNames:
        result, continuation_token = reviews(
            scrapePackageName,
            lang='en', # defaults to 'en'
            country='us', # defaults to 'us'
            sort=Sort.NEWEST, #sorting by new reviews, thus we get reviews based on latest update
            # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
            count=110, # defaults to 100
            # filter_score_with=5 # defaults to None(means all score)
        )

        # If you pass `continuation_token` as an argument to the reviews function at this point,
        # it will crawl the items after 3 review items.
        result, _ = reviews(
            scrapePackageName,
            continuation_token=continuation_token # defaults to None(load from the beginning)
        )
        #temp dataframe contains reviews per scrapePackageName
        tempReviews = pd.DataFrame(data={
            'Package name': scrapePackageName,
            'Reviewer name': map(lambda item: item['userName'], result),
            'Review': map(lambda item: item['content'], result),
            'Rating': map(lambda item: item['score'], result)
            })
        appReviews = pd.concat([appReviews, tempReviews])

        # Package name, Reviewer name, Review, Rating
        time.sleep(3) # Wait 3 seconds between each call; to prevent ipban from google scraping

    appReviews.to_csv((originalPackageName + '.csv'), index=False) #convert to CSV file

###################### Comment out these lines to fetch reviews
###################### If more than 1000 reviews, it will be limited from the scraper
get_reviews(['org.malwarebytes.antimalware', 'com.protectstar.spywaredetector', 'com.cyber_genius.cyber_tor', 'app.traced'], 'org.projectvoodoo.simplecarrieriqdetector')
# NOTE org.malwarebytes.antimalware and com.cyber_genius.cyber_tor has a limit of 1000 reviews
# ORIGINAL APP DOES NOT EXIST ON PLAYSTORE

get_reviews(['de.luhmer.owncloudnewsreader', 'com.ideashower.readitlater.pro', 'com.cubicdreams.feedmesh', 'com.devhd.feedly', 'jp.gocro.smartnews.android'], 'de.luhmer.owncloudnewsreader') 
# NOTE  'com.ideashower.readitlater.pro', 'com.cubicdreams.feedmesh', 'com.devhd.feedly' has a limit of 1000 reviews

get_reviews(['com.q4u.software', 'app.quantum.supdate', 'com.artemitsoftapp.appupdate', 'com.arumcomm.playservicesinfo'], 'fr.kwiatkowski.ApkTrack')
# NOTE  'app.quantum.supdate', 'com.artemitsoftapp.appupdate' has a limit of 1000 reviews
# ORIGINAL APP DOES NOT EXIST ON PLAYSTORE

get_reviews(['com.abcdjdj.rootverifier', 'com.joeykrim.rootcheck', 'com.jrummyapps.rootchecker', 'com.anu.developers3k.rootchecker', 'flar2.devcheck'], 'com.abcdjdj.rootverifier') 
# NOTE  'com.joeykrim.rootcheck', 'com.jrummyapps.rootchecker' has a limit of 1000 reviews

get_reviews(['com.paranoiaworks.unicus.android.sse', 'com.boxcryptor2.android', 'com.giraone.encmanlite', 'ch.protonmail.android'], 'name.seguri.android.isphoneencrypted')
# NOTE 'ch.protonmail.android' has a limit of 1000 reviews
# ORIGINAL APP DOES NOT EXIST ON PLAYSTORE
######################


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##### TASK 2 - Preprocess each review
I. Remove punctuations, <br>
II. Remove special characters and emojis, <br>
III. Turn numbers into text, <br>
IV. Remove extra white spaces, <br>
V. Turn all words into lowercase, <br>
VI. Remove stop words, <br>
VII. Lemmatize the reviews. <br>
VIII. Output 15 sample pre-processed reviews

In [ ]:
#### IMPORTING OUTSIDE LIBRARY FOR CONVERTING NUMBERS TO TEXT
%pip install num2words

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import stopwords
from nltk.corpus import stopwords
# Import lemmatizer; using WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from num2words import num2words

# numbers = {"1": "one", "2": "two", "3": "three", "4": "four", "5": "five", "6": "six", "7": "seven", "8": "eight", 9: "nine"}

#get stop words
stopwords_english = set(stopwords.words('english'))

#set lemmatizer
lemmatizer = WordNetLemmatizer()

# Preprocesses' reviews given a csv file name
# returns a dataFrame with all reviews preprocessed
def preprocessReviews(csvFileName):
    csvReviews = pd.read_csv(csvFileName + '.csv') # Read CSV File

    
    # for review in csvReviews['Review']:
    for i in csvReviews.index:
        review = csvReviews.at[i, 'Review']
        # remove punctuations, special characters, emojjis:
        try:
            newReview = re.sub('[^A-Za-z0-9\\s*]+', '', review) 
        except: # Exception can occur for empty review, e.g occured in name.seguri.android.isphoneencrypted.csv at line 486
            # remove current row, due to empty review causing the exception in regex
            csvReviews.drop([i])
            pass # in the case of empty review, skip as we removed the current row

        # convert all numbers to text
        tokens = nltk.word_tokenize(newReview)
        newReview = ""
        for token in tokens:
            try:
                newReview += num2words(token) + " "
            except:
                newReview += token + " "

        #remove all whitespaces 
        newReview = re.sub('[^A-Za-z0-9 ]+', '', newReview)

        #make all lowercase
        newReview = newReview.lower() 

        tokens = nltk.word_tokenize(newReview) # used tokenize to seperate the string into words, so we can apply lower() to it
        newReview = ""

        #remove stop words
        for token in tokens:
            if token not in stopwords_english: # current word is not a stop word; no changes to it
                newReview += token + " "
            else:
                pass # don't include the stopwords; tbh don't need an else statement

        tokens = nltk.word_tokenize(newReview) # used tokenize to seperate the string into words, so we can apply stem to it
        newReview = ""

        #lemmatize each word and then append back into the modified review
        for token in tokens:
            newReview += lemmatizer.lemmatize(token) + " "


        # Replace dataframe with the new preprocessed review
        csvReviews.at[i, 'Review'] = newReview
    
    # REMOVES ALL REVIEWS THAT HAVE AN EMPTY STRING
    csvReviews = csvReviews[csvReviews.Review != ""]
    csvReviews = csvReviews.reset_index(drop=True)

    #print 15 sample outputs for each review:
    for i in range(0, 15):
        print(csvReviews.at[i, 'Review'])
    print("")
    

    return csvReviews

print("ASSIGNED APP org.projectvoodoo.simplecarrieriqdetector...")
app1_df = preprocessReviews('org.projectvoodoo.simplecarrieriqdetector')

print("ASSIGNED APP name.seguri.android.isphoneencrypted...")
app2_df = preprocessReviews('name.seguri.android.isphoneencrypted')

print("ASSIGNED APP de.luhmer.owncloudnewsreader...")
app3_df = preprocessReviews('de.luhmer.owncloudnewsreader')

print("ASSIGNED APP fr.kwiatkowski.ApkTrack...")
app4_df = preprocessReviews('fr.kwiatkowski.ApkTrack')

print("ASSIGNED APP com.abcdjdj.rootverifier...")
app5_df = preprocessReviews('com.abcdjdj.rootverifier')
        


ASSIGNED APP org.projectvoodoo.simplecarrieriqdetector...
damn getting problem pop ad app solved really helping 
found trojan barcode scanner 
actually work pretty good worth download give 
help protect virus 
work 
nice 
awesome app 
great 
great 
upgraded premium wish option schedule scan doe multiple time day security news bit outdated wish update button dashboard early fully judge detail yet 
effective realtime working app really helped alot getting rid virus phone one issue app tooked much time expectation starting next time working fast 
good app fast working 
started use easy use 
world window computer malwarebytes capable rooting pesky little bug straggler antivirus always malwarebytes reign 
picked ransomware phone installed app thank mb 

ASSIGNED APP name.seguri.android.isphoneencrypted...
option wipeoverwrite garbage free space needed make file deleted outside app unrecoverable 
fantastic app could vault feature file automatically encrypted added vault 
cant open encrypted 

#### BONUS of TASK 2 (5 Points): NLTK has a list of stop words which can be customized or replaced. Does the choice of stop words has any impact on the analysis of the reviews? What is the risk of using a non-customized list of stop words in analyzing mobile app reviews for software requirements elicitation?

The choice of stop words has a huge impact on the analysis of reviews, if stop words gets too customized we can lose valuable data due to context of reviews being changed heavily to the point where the review can mean completly something else. i.e putting 'good' as a stop word can change the meaning of e.g 'this app is not good as X competitor app', if we customized stop words to include 'not', a negative review can be seen as a positive one.
<br>
The risk of using non-customized list of stop words in analyzing reviews for elicitation can possibly allow repetition and waste of computing and data resources. We can still easily capture the sentiment of a review without the need of some words e.g 'the app was great!' we can have 'the' removed and still get the same sentiment.

#### TASK 3: SENTIMENT ANALYSIS

In [ ]:
# Pip install libraries (not included from anaconda)
%pip install textblob
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


I. Use Textblob for calculating Reviews Sentiment; <br>
II. The output should be a table with headers in the format of <App’s package name,
Review, Polarity>. 

In [ ]:
from textblob import TextBlob

# Using TextBlob to get sentiment
def get_textBlobSentiment(reviews_dataFrame):
    # # <App’s package name, Review, Polarity>
    # print("<App’s package name, Review, Polarity>")
    
    # # for review in reviews_dataFrame['Review']:
    # # NOTE printing only 250 lines, due to notebook lagging
    # for i in range(0,250):
    #     review = reviews_dataFrame.at[i, 'Review']
    #     packageName = reviews_dataFrame.at[i, 'Package name']
    #     print(packageName + ", " + review + ", " + str((TextBlob(review).sentiment)[0]))
    
    #### CHANGED TO USING PANDAS TABLE, incase of 'bad format'
    df = reviews_dataFrame.rename({'Package name': "App's package name",}, axis=1)
    df = df.drop('Reviewer name', axis=1) #1 for axis
    df = df.drop('Rating', axis=1)

    df['Polarity'] = df.apply(lambda row: str((TextBlob(row['Review']).sentiment)[0]), axis = 1)

    # display(df)
    display(df.head(250)) # DISPLAYS first 250 rows for each app group, would display more but notebook is lagging

pd.set_option('display.max_rows', 250)
get_textBlobSentiment(app1_df)
get_textBlobSentiment(app2_df)
get_textBlobSentiment(app3_df)
get_textBlobSentiment(app4_df)
get_textBlobSentiment(app5_df)

,App's package name,Review,Polarity
0,org.malwarebytes.antimalware,damn getting problem pop ad app solved really helping,0.2
1,org.malwarebytes.antimalware,found trojan barcode scanner,0.0
2,org.malwarebytes.antimalware,actually work pretty good worth download give,0.3125
3,org.malwarebytes.antimalware,help protect virus,0.0
4,org.malwarebytes.antimalware,work,0.0
5,org.malwarebytes.antimalware,nice,0.6
6,org.malwarebytes.antimalware,awesome app,1.0
7,org.malwarebytes.antimalware,great,0.8
8,org.malwarebytes.antimalware,great,0.8
9,org.malwarebytes.antimalware,upgraded premium wish option schedule scan doe multiple time day security news bit outdated wish update button dashboard early fully judge detail yet,-0.10000000000000002


,App's package name,Review,Polarity
0,com.paranoiaworks.unicus.android.sse,option wipeoverwrite garbage free space needed make file deleted outside app unrecoverable,0.2
1,com.paranoiaworks.unicus.android.sse,fantastic app could vault feature file automatically encrypted added vault,0.4
2,com.paranoiaworks.unicus.android.sse,cant open encrypted folder without decrypting permanently want open encrypted folder add new file want folder encrypted adding file,0.04545454545454545
3,com.paranoiaworks.unicus.android.sse,ive started use came across issue immediately setting wipe source encryption encrypted excel file source file deleted great later decrypted file opened excel app needed closed excel surprised later find unencrypted file still folder alongside previous encrypted one depending whether source file changed necessary either delete manually reencrypt delete source part process would hoped app would automatically reencrypt delete source file least option setup,0.04166666666666668
4,com.paranoiaworks.unicus.android.sse,hi could please allow file name encrytion generate new random file name,-0.18181818181818182
5,com.paranoiaworks.unicus.android.sse,ask term file security,0.0
6,com.paranoiaworks.unicus.android.sse,fabulous app open source incredibly intuitive file easy find interface user outstanding encrypts folder file encrypts item sd card instruction readable easy find top password vault password generator way encrypt text select communication app use also wipe file though would nice link wiper shredder tool window finally even give button shut app dont android apps feature thanks incredibly useful app,0.38787878787878793
7,com.paranoiaworks.unicus.android.sse,useful simple use love intuitive clear ui please dont spoil material design keep offline also using window version laptop android four hundred twentytwo,0.225
8,com.paranoiaworks.unicus.android.sse,decrypt file always say file corrupted invalid password uninstalledreinstalled three time still thing garbage,0.0
9,com.paranoiaworks.unicus.android.sse,ill stick current encryption app much easier use,-0.09999999999999999


,App's package name,Review,Polarity
0,de.luhmer.owncloudnewsreader,update dev able fix issue reddit feed imgur page seem buggy often jump external browser nexus five seventytwo thousand thirteen 9android six hundred one,0.25
1,de.luhmer.owncloudnewsreader,great app,0.8
2,de.luhmer.owncloudnewsreader,work great customise feed colour option would great though ie bbc news dark blue id like change bright red,0.3583333333333334
3,de.luhmer.owncloudnewsreader,json error,0.0
4,de.luhmer.owncloudnewsreader,thanks great app use daily sync even work reliably home server slowest adsl money buy dynamic ip address nonstandard port self signed certificate,0.25
5,de.luhmer.owncloudnewsreader,frequently crash playing pod cast,0.1
6,de.luhmer.owncloudnewsreader,disabling host name verification fixed login problem,0.1
7,de.luhmer.owncloudnewsreader,love,0.5
8,de.luhmer.owncloudnewsreader,good indeed,0.7
9,de.luhmer.owncloudnewsreader,need setup owncloud server,0.0


,App's package name,Review,Polarity
0,com.q4u.software,bye,0.0
1,com.q4u.software,cool app,0.35
2,com.q4u.software,fantastic,0.4
3,com.q4u.software,nice slow,0.14999999999999997
4,com.q4u.software,impressive,1.0
5,com.q4u.software,e,0.0
6,com.q4u.software,good,0.7
7,com.q4u.software,excellent,1.0
8,com.q4u.software,rate app,0.0
9,com.q4u.software,im confortable apps,0.0


,App's package name,Review,Polarity
0,com.abcdjdj.rootverifier,good app,0.7
1,com.abcdjdj.rootverifier,good app love,0.6
2,com.abcdjdj.rootverifier,great root checker program,0.8
3,com.abcdjdj.rootverifier,easy install,0.43333333333333335
4,com.abcdjdj.rootverifier,best root checker,1.0
5,com.abcdjdj.rootverifier,good,0.7
6,com.abcdjdj.rootverifier,good apps,0.7
7,com.abcdjdj.rootverifier,good,0.7
8,com.abcdjdj.rootverifier,awesome,1.0
9,com.abcdjdj.rootverifier,perfect perfect,1.0


III. Use Vader for calculating Reviews Sentiment;<br>
IV. The output should be a table with headers in the format of <App’s package name,
Review, Polarity >

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Using Vader to get sentiment
def get_vaderSentiment(reviews_dataFrame):
    # # <App’s package name, Review, Polarity>
    # print("<App’s package name, Review, Polarity>")
    
    # # for review in reviews_dataFrame['Review']:
    # # NOTE printing only 250 lines, due to notebook lagging
    # for i in range(0,250):
    #     review = reviews_dataFrame.at[i, 'Review']
    #     packageName = reviews_dataFrame.at[i, 'Package name']
    #     print(packageName + ", " + review + ", " + str((analyzer.polarity_scores(review))['compound']))

    #### CHANGED TO USING PANDAS TABLE, incase of 'bad format'
    df = reviews_dataFrame.rename({'Package name': "App's package name",}, axis=1)
    df = df.drop('Reviewer name', axis=1) #1 for axis
    df = df.drop('Rating', axis=1)

    df['Polarity'] = df.apply(lambda row: str((analyzer.polarity_scores(row['Review'])['compound'])), axis = 1)

    # display(df)
    display(df.head(250)) # DISPLAYS first 250 rows for each app group, would display more but notebook is lagging
    
# get vader sentiment's polarity
get_vaderSentiment(app1_df)
get_vaderSentiment(app2_df)
get_vaderSentiment(app3_df)
get_vaderSentiment(app4_df)
get_vaderSentiment(app5_df)


,App's package name,Review,Polarity
0,org.malwarebytes.antimalware,damn getting problem pop ad app solved really helping,-0.204
1,org.malwarebytes.antimalware,found trojan barcode scanner,0.0
2,org.malwarebytes.antimalware,actually work pretty good worth download give,0.7906
3,org.malwarebytes.antimalware,help protect virus,0.6486
4,org.malwarebytes.antimalware,work,0.0
5,org.malwarebytes.antimalware,nice,0.4215
6,org.malwarebytes.antimalware,awesome app,0.6249
7,org.malwarebytes.antimalware,great,0.6249
8,org.malwarebytes.antimalware,great,0.6249
9,org.malwarebytes.antimalware,upgraded premium wish option schedule scan doe multiple time day security news bit outdated wish update button dashboard early fully judge detail yet,0.7783


,App's package name,Review,Polarity
0,com.paranoiaworks.unicus.android.sse,option wipeoverwrite garbage free space needed make file deleted outside app unrecoverable,0.5106
1,com.paranoiaworks.unicus.android.sse,fantastic app could vault feature file automatically encrypted added vault,0.5574
2,com.paranoiaworks.unicus.android.sse,cant open encrypted folder without decrypting permanently want open encrypted folder add new file want folder encrypted adding file,0.0201
3,com.paranoiaworks.unicus.android.sse,ive started use came across issue immediately setting wipe source encryption encrypted excel file source file deleted great later decrypted file opened excel app needed closed excel surprised later find unencrypted file still folder alongside previous encrypted one depending whether source file changed necessary either delete manually reencrypt delete source part process would hoped app would automatically reencrypt delete source file least option setup,0.9485
4,com.paranoiaworks.unicus.android.sse,hi could please allow file name encrytion generate new random file name,0.4939
5,com.paranoiaworks.unicus.android.sse,ask term file security,0.34
6,com.paranoiaworks.unicus.android.sse,fabulous app open source incredibly intuitive file easy find interface user outstanding encrypts folder file encrypts item sd card instruction readable easy find top password vault password generator way encrypt text select communication app use also wipe file though would nice link wiper shredder tool window finally even give button shut app dont android apps feature thanks incredibly useful app,0.9725
7,com.paranoiaworks.unicus.android.sse,useful simple use love intuitive clear ui please dont spoil material design keep offline also using window version laptop android four hundred twentytwo,0.8885
8,com.paranoiaworks.unicus.android.sse,decrypt file always say file corrupted invalid password uninstalledreinstalled three time still thing garbage,0.0
9,com.paranoiaworks.unicus.android.sse,ill stick current encryption app much easier use,0.0


,App's package name,Review,Polarity
0,de.luhmer.owncloudnewsreader,update dev able fix issue reddit feed imgur page seem buggy often jump external browser nexus five seventytwo thousand thirteen 9android six hundred one,0.0
1,de.luhmer.owncloudnewsreader,great app,0.6249
2,de.luhmer.owncloudnewsreader,work great customise feed colour option would great though ie bbc news dark blue id like change bright red,0.9274
3,de.luhmer.owncloudnewsreader,json error,-0.4019
4,de.luhmer.owncloudnewsreader,thanks great app use daily sync even work reliably home server slowest adsl money buy dynamic ip address nonstandard port self signed certificate,0.8625
5,de.luhmer.owncloudnewsreader,frequently crash playing pod cast,-0.2263
6,de.luhmer.owncloudnewsreader,disabling host name verification fixed login problem,-0.7003
7,de.luhmer.owncloudnewsreader,love,0.6369
8,de.luhmer.owncloudnewsreader,good indeed,0.4404
9,de.luhmer.owncloudnewsreader,need setup owncloud server,0.0


,App's package name,Review,Polarity
0,com.q4u.software,bye,0.0
1,com.q4u.software,cool app,0.3182
2,com.q4u.software,fantastic,0.5574
3,com.q4u.software,nice slow,0.4215
4,com.q4u.software,impressive,0.5106
5,com.q4u.software,e,0.0
6,com.q4u.software,good,0.4404
7,com.q4u.software,excellent,0.5719
8,com.q4u.software,rate app,0.0
9,com.q4u.software,im confortable apps,0.0


,App's package name,Review,Polarity
0,com.abcdjdj.rootverifier,good app,0.4404
1,com.abcdjdj.rootverifier,good app love,0.7964
2,com.abcdjdj.rootverifier,great root checker program,0.6249
3,com.abcdjdj.rootverifier,easy install,0.4404
4,com.abcdjdj.rootverifier,best root checker,0.6369
5,com.abcdjdj.rootverifier,good,0.4404
6,com.abcdjdj.rootverifier,good apps,0.4404
7,com.abcdjdj.rootverifier,good,0.4404
8,com.abcdjdj.rootverifier,awesome,0.6249
9,com.abcdjdj.rootverifier,perfect perfect,0.8126


V. How does the sentiments retrieved by Textblob and Vader compare with each
other? How do you interpret the similarity/difference? Which one is the best option
for review analysis of your apps? Why?<br><br>

From an overlook Vader seems to be more accurate in comparison to Textblob. Just taking a quick glance at the values we see that Textblob has less tolerance (less weight) and gives out 1s and -1s often when it sees an obvious positive or negative. Whereas Vader has more tolerance (more weight) when it depicts sentiments thus in that sense Vader can be used to distinguish reviews and Textblob can be used to identify the obvious positive or negative sentiments.
<br><br>
To interpret the simalrity or difference one must compare the same text input and look at the polarity sentiment values. In terms of similarity both ranges values between -1 being negative and +1 being positive sentiment. The difference being Vader also gives seperate values on positive, negative and neutral which can be useful when it comes to distinguish similar sentiments, i.e comparing 2 positive sentiments to see which one has a more positive polarity to it. Whereas TextBlob is more likely to get 1 to both if they are an obvious positive statement, i.e in the above reviews TextBlob gave 'app give best protection computer help never problem android dont go life without' a polarity of 1 and also gave 1 to 'best'. Whereas Vader gave 0.8465 to the first review, and 0.6369 for 'best'.
<br><br>
Thus, we can see that Vader is the better option for reviewing the comments. Reason being, it attempts to be more accurate by differianting positive reviews i.e in the above example the more descriptive and positive toning one had a higher score than the simple review, 'best'. This can be useful when it comes to figuring out which reviews to take a look at for eliciting requirements, i.e lower (negative) or higher (positive) scores are typically descriptive and provides a good tone to take a look at. Wheras Textblob might be useful when it comes to having a quick glance on how well an app is doing.

#### TASK 4: TOPIC MODELING Using LDA
I. Among your five ASSIGNED APPs, choose the one with the highest number of
reviews.<br>
II. From its reviews, extract minimum of 10 LDA topics [1, 2] each being described by
at least 7 Words.<br>


In [ ]:
# install genism for topic modeling using LDA
# Pip install libraries (not included from anaconda)
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gensim
import gensim.corpora as corpora

# NOTE The chosen ASSIGNED APP is com.abcdjdj.rootverifier, I only had 2 apps on the playstore and this has the most
# It only has 110 reviews. The reviews are stored within variable app5_df (from earlier: app5_df = preprocessReviews('com.abcdjdj.rootverifier'))

dict_tokenizedAppReviews = {} # this will contain all all the reviews in tokenized format, for the ASSIGNED APP, its 3 competitors and 1 similar

for i in app5_df.index:
    review = app5_df.at[i,'Review']
    review = nltk.word_tokenize(review)

    if app5_df.at[i,'Package name'] in dict_tokenizedAppReviews: # package already in dict_tokenizedAppReviews, append to its value
        dict_tokenizedAppReviews[app5_df.at[i,'Package name']].append(review)

    else: # new package found, create new key with inital review
        dict_tokenizedAppReviews[app5_df.at[i,'Package name']] = [review]

# print(dict_tokenizedAppReviews['com.abcdjdj.rootverifier'])

# dict_lda_models = {} # this will contain all lda models, i.e lda model for ASSIGNED APP, it's 3 competitors and 1 similar app

# for app in dict_tokenizedAppReviews:
def get_LDAModel(appPackageName, numberOfTopics):
    tokenized_assignedApp_reviews = dict_tokenizedAppReviews[appPackageName] # contains tokenized app reviews for current app

    # id tag words & frequency via corpus
    id2word = corpora.Dictionary(tokenized_assignedApp_reviews)
    corpus = [id2word.doc2bow(text) for text in tokenized_assignedApp_reviews]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=numberOfTopics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
    # lda_model.print_topics()
    # dict_lda_models[app] = lda_model
    return lda_model


def printTopicsTable(topics):
    dataF = pd.DataFrame(map(lambda topic: (topic[1]), topics))
    dataF.columns = ["Topics"]

    pd.set_option('display.max_colwidth', None)
    display(dataF)

dict_containLDAModels = {}
dict_containLDAModels['com.abcdjdj.rootverifier'] = get_LDAModel('com.abcdjdj.rootverifier', 10)
printTopicsTable(dict_containLDAModels['com.abcdjdj.rootverifier'].print_topics(num_words=7))
## Print statements to verify:
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]
# print(corpus[:1])

,Topics
0,"0.215*""rooted"" + 0.151*""great"" + 0.137*""phone"" + 0.079*""know"" + 0.063*""mobile"" + 0.063*""cell"" + 0.005*""isnt"""
1,"0.138*""device"" + 0.087*""xhdpi"" + 0.062*""nice"" + 0.054*""say"" + 0.049*""app"" + 0.044*""hdpi"" + 0.044*""graphic"""
2,"0.031*""root"" + 0.027*""app"" + 0.019*""say"" + 0.019*""box"" + 0.019*""kill"" + 0.019*""great"" + 0.019*""love"""
3,"0.097*""sure"" + 0.093*""good"" + 0.092*""im"" + 0.062*""work"" + 0.037*""root"" + 0.017*""checker"" + 0.012*""version"""
4,"0.036*""hack"" + 0.025*""doo"" + 0.025*""root"" + 0.014*""busy"" + 0.014*""box"" + 0.014*""apps"" + 0.014*""saying"""
5,"0.337*""good"" + 0.016*""checking"" + 0.016*""job"" + 0.016*""exactly"" + 0.010*""root"" + 0.009*""access"" + 0.009*""simpler"""
6,"0.082*""work"" + 0.057*""hate"" + 0.054*""straight"" + 0.054*""point"" + 0.054*""sab"" + 0.054*""nit"" + 0.054*""abhi"""
7,"0.091*""one"" + 0.086*""thing"" + 0.051*""easy"" + 0.026*""app"" + 0.021*""quick"" + 0.021*""use"" + 0.016*""like"""
8,"0.102*""app"" + 0.095*""work"" + 0.082*""awesome"" + 0.082*""cool"" + 0.082*""check"" + 0.082*""even"" + 0.070*""maddy"""
9,"0.208*""look"" + 0.107*""five"" + 0.105*""star"" + 0.019*""told"" + 0.019*""busybox"" + 0.013*""didnt"" + 0.013*""app"""


#### TASK 4 Contd.
III. How does these review topics (i.e. summarized user feedback) relate to the
software features (functionalities) you extracted from app description in
Assignment 2 (manual) and/or Assignment 3 (automated bi-grams/tri-grams)?<br>

#### Answer:
Majority of the words in the review topics are surprisingly related to both the app description and the automated bigrams. I found this surprising as more than half the reviews from this app were a few words or even a single word review. Although this app had a basic description in which the functionalities were simpily: 
<li> Check if device is rooted
<li> RootVerifier checks if your Android device has Superuser ("root") access or not 

<br>The very first topic contained "rooted phone/mobile/cell great/perfect app" which is essentially something to do with rooting and phones. The second topic explicitly has the words "check root app free apk download", these 2 topics essentially state the functionality, which is to check/verify if a phone is rooted while being free.
<br>As for the automated bi-grams they were similar in the sense its the same terms but nothing coherrent as when comparing the description to the topics.
<br>Although some of the review topics weren't accurate in the means of being concrete as some of the topics had some unrelated things such as terms like "maddy, b, sab, nit" which is probably caused from spelling/grammar mistakes or the use of other languages but written using english letters. Overall it has been a great insight.

#### TASK 4 Contd.
IV. For this chosen ASSIGNED APP, extract topics of reviews (minimum of 5 Topics
each described by 7 Words) for each of the competitor and similar apps
(separately).<br>

In [ ]:
# 5 topics and each described with 7 words
dict_containLDAModels['com.joeykrim.rootcheck'] = get_LDAModel('com.joeykrim.rootcheck', 5)
print("Competitor app com.joeykrim.rootcheck:")
printTopicsTable(dict_containLDAModels['com.joeykrim.rootcheck'].print_topics(num_words=7))

dict_containLDAModels['com.jrummyapps.rootchecker'] = get_LDAModel('com.jrummyapps.rootchecker', 5)
print("Competitor app com.jrummyapps.rootchecker:")
printTopicsTable(dict_containLDAModels['com.jrummyapps.rootchecker'].print_topics(num_words=7))

dict_containLDAModels['com.anu.developers3k.rootchecker'] = get_LDAModel('com.anu.developers3k.rootchecker', 5)
print("Competitor app com.anu.developers3k.rootchecker:")
printTopicsTable(dict_containLDAModels['com.anu.developers3k.rootchecker'].print_topics(num_words=7))

dict_containLDAModels['flar2.devcheck'] =get_LDAModel('flar2.devcheck', 5)
print("Similar app flar2.devcheck:")
printTopicsTable(dict_containLDAModels['flar2.devcheck'].print_topics(num_words=7))

# print(tabulate(dataF, headers = 'keys', tablefmt = 'psql'))


Competitor app com.joeykrim.rootcheck:


,Topics
0,"0.218*""app"" + 0.088*""root"" + 0.055*""best"" + 0.027*""like"" + 0.023*""dont"" + 0.021*""use"" + 0.020*""checker"""
1,"0.164*""nice"" + 0.075*""phone"" + 0.027*""application"" + 0.019*""download"" + 0.018*""awesome"" + 0.015*""service"" + 0.014*""note"""
2,"0.064*""useful"" + 0.048*""great"" + 0.043*""rooted"" + 0.032*""check"" + 0.028*""love"" + 0.023*""device"" + 0.021*""know"""
3,"0.246*""good"" + 0.063*""work"" + 0.037*""bad"" + 0.018*""properly"" + 0.018*""really"" + 0.014*""please"" + 0.013*""access"""
4,"0.038*""ok"" + 0.035*""apps"" + 0.033*""ad"" + 0.027*""thank"" + 0.027*""super"" + 0.023*""waste"" + 0.017*""time"""


Competitor app com.jrummyapps.rootchecker:


,Topics
0,"0.158*""good"" + 0.031*""need"" + 0.030*""like"" + 0.027*""cool"" + 0.025*""awesome"" + 0.024*""jrummy"" + 0.020*""app"""
1,"0.059*""checker"" + 0.050*""root"" + 0.021*""much"" + 0.021*""helpful"" + 0.020*""android"" + 0.017*""ur"" + 0.016*""rooted"""
2,"0.056*""love"" + 0.041*""use"" + 0.025*""easy"" + 0.023*""four"" + 0.022*""ad"" + 0.018*""two"" + 0.016*""time"""
3,"0.060*""rooted"" + 0.044*""work"" + 0.044*""device"" + 0.042*""phone"" + 0.028*""apps"" + 0.021*""know"" + 0.020*""one"""
4,"0.174*""app"" + 0.084*""root"" + 0.065*""great"" + 0.059*""nice"" + 0.045*""best"" + 0.036*""u"" + 0.032*""check"""


Competitor app com.anu.developers3k.rootchecker:


,Topics
0,"0.057*""rooted"" + 0.041*""app"" + 0.040*""show"" + 0.029*""device"" + 0.023*""root"" + 0.018*""result"" + 0.018*""great"""
1,"0.072*""app"" + 0.026*""well"" + 0.026*""phone"" + 0.026*""rooted"" + 0.026*""nice"" + 0.014*""useful"" + 0.014*""simple"""
2,"0.030*""great"" + 0.029*""nice"" + 0.029*""easy"" + 0.023*""informative"" + 0.015*""recommend"" + 0.015*""3k"" + 0.015*""highly"""
3,"0.049*""root"" + 0.042*""app"" + 0.022*""detail"" + 0.022*""find"" + 0.022*""excellent"" + 0.015*""false"" + 0.015*""give"""
4,"0.141*""good"" + 0.024*""detail"" + 0.013*""bin"" + 0.013*""su"" + 0.013*""finder"" + 0.013*""guy"" + 0.013*""true"""


Similar app flar2.devcheck:


,Topics
0,"0.069*""good"" + 0.039*""nice"" + 0.038*""useful"" + 0.023*""app"" + 0.020*""work"" + 0.019*""sensor"" + 0.018*""system"""
1,"0.029*""thanks"" + 0.017*""see"" + 0.016*""update"" + 0.014*""running"" + 0.013*""memory"" + 0.013*""help"" + 0.012*""status"""
2,"0.023*""battery"" + 0.016*""also"" + 0.015*""im"" + 0.015*""four"" + 0.012*""looking"" + 0.012*""click"" + 0.012*""wrong"""
3,"0.138*""app"" + 0.062*""great"" + 0.030*""info"" + 0.027*""device"" + 0.023*""show"" + 0.023*""one"" + 0.021*""information"""
4,"0.028*""need"" + 0.027*""pro"" + 0.026*""version"" + 0.025*""everything"" + 0.016*""monitor"" + 0.016*""please"" + 0.015*""know"""


#### TASK 4 Contd.

V. How does the topics compare to the topics extracted from reviews of your
ASSIGNED APP? 

A basic overlook via manual inspection shows that the topics do relate with the ASSIGNED APP, especially the competitor ones.
ASSIGNED APP contained the following: <br>
0	0.215*"rooted" + 0.151*"great" + 0.137*"phone" + 0.079*"know" + 0.063*"mobile" + 0.063*"cell" + 0.005*"isnt" <br>
1	0.138*"device" + 0.087*"xhdpi" + 0.062*"nice" + 0.054*"say" + 0.049*"app" + 0.044*"hdpi" + 0.044*"graphic" <br>
2	0.031*"root" + 0.027*"app" + 0.019*"say" + 0.019*"box" + 0.019*"kill" + 0.019*"great" + 0.019*"love" <br>
3	0.097*"sure" + 0.093*"good" + 0.092*"im" + 0.062*"work" + 0.037*"root" + 0.017*"checker" + 0.012*"version" <br>
4	0.036*"hack" + 0.025*"doo" + 0.025*"root" + 0.014*"busy" + 0.014*"box" + 0.014*"apps" + 0.014*"saying" <br>
5	0.337*"good" + 0.016*"checking" + 0.016*"job" + 0.016*"exactly" + 0.010*"root" + 0.009*"access" + 0.009*"simpler" <br>
6	0.082*"work" + 0.057*"hate" + 0.054*"straight" + 0.054*"point" + 0.054*"sab" + 0.054*"nit" + 0.054*"abhi" <br>
7	0.091*"one" + 0.086*"thing" + 0.051*"easy" + 0.026*"app" + 0.021*"quick" + 0.021*"use" + 0.016*"like" <br>
8	0.102*"app" + 0.095*"work" + 0.082*"awesome" + 0.082*"cool" + 0.082*"check" + 0.082*"even" + 0.070*"maddy" <br>
9	0.208*"look" + 0.107*"five" + 0.105*"star" + 0.019*"told" + 0.019*"busybox" + 0.013*"didnt" + 0.013*"app" <br>

Let's choose the first topic: <br>
0	0.215*"rooted" + 0.151*"great" + 0.137*"phone" + 0.079*"know" + 0.063*"mobile" + 0.063*"cell" + 0.005*"isnt"

Upon manual inspection e.g Competitor app com.joeykrim.rootcheck <br>
2	0.064*"useful" + 0.048*"great" + 0.043*"rooted" + 0.032*"check" + 0.028*"love" + 0.023*"device" + 0.021*"know"<br>
Both had the same keywords such as rooted, device (can be the same as phone, mobile cell), other sentiment words such as great is found in both, 
likewise with the rest,<br><br> Competitor app com.jrummyapps.rootchecker: <br>
3	0.060*"rooted" + 0.044*"work" + 0.044*"device" + 0.042*"phone" + 0.028*"apps" + 0.021*"know" + 0.020*"one"<br>
Also has important key words such as rooted, device, phone that is also found with the rest. <br><br>

Competitor app com.anu.developers3k.rootchecker: <br>
0	0.057*"rooted" + 0.041*"app" + 0.040*"show" + 0.029*"device" + 0.023*"root" + 0.018*"result" + 0.018*"great" <br>
Keywords similar: rooted, app, device, root, great <br><

The only exception was the similar app, as you can see above only some keywords such as app, great, device is shown but keywords such as rooted or root isn't found. The reason being because the similar app was chosen because it involved similar functionalities such as inspection of device, i.e hardware in this case and wasn't directly related to rooting. Which is why it's not as similar as the rest of the apps. <br>
Whereas the competitors and the ASSIGNED APP nearly have the same keywords/topics and the words that didn't match had a similar word but as an synynom or random words that have been mistakenly chosen, e.g: <br>
6	0.082*"work" + 0.057*"hate" + 0.054*"straight" + 0.054*"point" + 0.054*"sab" + 0.054*"nit" + 0.054*"abhi" <br>
This was fetched from the ASSIGNED APP and as you can see some of the words such as 'sab, nit, abhi' don't correlate as these are probably spelling mistakes or a different language.

<br> The code fragment below was an attempt to find cosine simalarities based on topics that seemed similar to the ASSIGNED APP and it's competitors. By just attempting with random topics chosen in each competitor/similar vs ASSIGNED we can clearly see that competitor apps have a higher similarity to the ASSIGNED APP based on the values.
<br> In conclusion, the topics with competitor apps and ASSIGNED APPs have more similarities than discrepancies, as upon manual inspection we see that majority of the keyterms are identical and/or are synonyms. And this makes sense as the competitor apps are also verifying phones are rooted with bonus features (this was noted in lab2).

In [ ]:
# import gensim.matutils.cossim as cossim

# id tag words & frequency via corpus
# id2word = corpora.Dictionary(dict_tokenizedAppReviews['com.joeykrim.rootcheck'])
# corpus = [id2word.doc2bow(text) for text in dict_tokenizedAppReviews['com.joeykrim.rootcheck']]

# id2word2 = corpora.Dictionary(dict_tokenizedAppReviews['com.jrummyapps.rootchecker'])
# corpus2 = [id2word2.doc2bow(text) for text in dict_tokenizedAppReviews['com.jrummyapps.rootchecker']]

# doc1 = dict_containLDAModels['com.abcdjdj.rootverifier'].get_document_topics(corpus[0], minimum_probability=0)
# doc2 = dict_containLDAModels['com.abcdjdj.rootverifier'].get_document_topics(corpus2[0], minimum_probability=0)

# each competitve app and similarity app was compared with their topic terms and then used cosine similarity to calculate how similar they are to one another
doc1 = sorted(dict_containLDAModels['com.abcdjdj.rootverifier'].get_topic_terms(0, topn=7))
doc2 = sorted(dict_containLDAModels['com.joeykrim.rootcheck'].get_topic_terms(1, topn=7))
print(gensim.matutils.cossim(doc1, doc2))

doc1 = sorted(dict_containLDAModels['com.abcdjdj.rootverifier'].get_topic_terms(2, topn=7))
doc2 = sorted(dict_containLDAModels['com.jrummyapps.rootchecker'].get_topic_terms(0, topn=7))
print(gensim.matutils.cossim(doc1, doc2))

doc1 = sorted(dict_containLDAModels['com.abcdjdj.rootverifier'].get_topic_terms(2, topn=7))
doc2 = sorted(dict_containLDAModels['com.anu.developers3k.rootchecker'].get_topic_terms(2, topn=7))
print(gensim.matutils.cossim(doc1, doc2))

doc1 = sorted(dict_containLDAModels['com.abcdjdj.rootverifier'].get_topic_terms(0, topn=7))
doc2 = sorted(dict_containLDAModels['flar2.devcheck'].get_topic_terms(2, topn=7))
print(gensim.matutils.cossim(doc1, doc2))


0.27382401218429914
0.34571609978488066
0.2163637065287548
0.18938007426573855


#### BONUS for TASK 4
I. Retrieve the topic for each review of your ASSIGNED APP. This should be done
by finding the topic that has the highest percentage contribution in that Review.  <br>
II. Provide a list of 10 Reviews that you believe developers should take care of with
high priority. Discuss your logic for this selection. <br>
III. Add the code and the discussion under “BONUS for TASK 4” in your Notebook

#### Answer II:
Please note that the app I choose for this task is `om.abcdjdj.rootverifier`. Reason being out of the 2 ASSIGNED APPs that had reviews, this had the most with 109 reviews. So it was particulary difficult to find 10 reviews that the developer needs to take care of with. But what was interesting to find is majority of the reviews listed below have the same dominant topic which was #7, it seems like this topic is more related to negative type reviews or to criticize.<br> 
The topic has the keywords: `one, thing, easy, app, quick, use, like, star, great, working` <br>
And these keywords are synonymous with negative reviews due to haveing key words that can be used in terms of e.g "one star", not "working", hard to "use" etc.
<br><br>
List of 10 reviews that I believe developers should take care of with are listed below (provided table row number along with dominant topic, and percentage):<br><br>
11	7.0	0.8897  would give zero star id could<br>
The logic that was used to choose this review as this was the only blatant review stating their hatred towards the app without stating a reason as to why. The developer(s) can further engage with this user by replying what caused them to wanting to rate the app as a 0 star. By doing so they can figure out where to improve the app inorder to increase satisfaction from its user base.
<br><br>
38	9.0	0.9544  compared apps space theyre given 860kb featureless app going get five star dev want look competition uninstalling<br>
This user wants a root verifying app to take less space on their phone. As such the developer(s) should attempt to make the app gradle files take up less space as more space can cause disatisfaction to users who care about their phone memory. The developer can either attempt to shorten code etc. without losing features or create a seperate product line i. a lite version of the app, similar to what Facebook did with the messenger app, created a lite messenger app to take up far less space.
<br><br>
49	7.0	0.8774	want something root tab please actually never rooted device one star isnt app thanks <br>
The developers should clearly state that their app is only meant to verify root as some users might not understand, this can be done by improving UI and/or improving app description.
<br><br>
50	7.0	0.9411	keep simple stupid old bloke like retired ncr still making mainframe right <br>
This review wants the app to be 'simple', reason why I chose this is so developers can try to improve the UI (i.e usability of the app). This can also been as a way for developer(s) can improve their app by having a section in which they educate how rooting works etc.
<br><br>
53	7.0	0.9490	great ui improve app l suggest make tweak calculation full function handy app dout <br>
This review is very important as its one of the reviews they are trying to give a suggestion to a new feature or improvement to the developer. Following reviews such as these that are within the scope of the project/product can be very beneficial to keep user satisfaction/
<br><br>
72	5.0	0.7965	cant open file <br>
Bug issue, developer can request their android version and device and attempt to fix it for user satisfaction.
<br><br>
76	5.0	0.5963	wont work force close vendettasoft better root checker <br>
Bug issue again, developer can further attempt to contact the user and maybe even take some feature ideas from the software mentioned from the user.
<br><br>
81	3.0	0.8920	downloaded busy box nook tablet stay welcome screen busy box didnt root privelages pop dont know root authority im sure great apk cant seem figure <br>
Another bug issue in which the app was stuck on welcome screen and they couldn't figure out how to verify root privileges. This can either be a bug or UI is difficult to understand, this is important to the developer(s) as it can allow for improvement to their ui or figure out an bug case to increase user satisfaction.
<br><br>
93	2.0	0.9876	permission necessary say phone isnt root busy box say bug shelled remount change permission wes clever great root package cloud terminal emulator shellconsolse root browser root tool box rom manager su elite cpu system tuner great app love nopermissions kill adb three thousand nine hundred sixtythree l echo remount fix perm read write root kill l exit xda er lulz lot kill lulz irl exit five <br>
This seems like an extensive bug report. Given that there's so many bug reports the developers should also look into ways of creating a feedback system within the app, or clearly stating where to contact within the app (i.e email of sorts).
<br><br>
99	2.0	0.9358	love app view busybox detail hope next update put view su detail <br>
This user is giving a suggestion to the developer(s) to add a detail section for a part of the app, which ofcourse is benefitcial to the developer(s) as it can increase user satisfaction.
<br><br>


In [ ]:
# map dominant topic based on each review
def get_dominantTopics(ldamodel, corpus, texts):
    # this dataframe will hold each review mapped to its dominant topic
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True) #sort reverse order per row in LDAMODEL based on given corpus
        # Get the Dominant topic, Percent Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                # sent_topics_df = sent_topics_df.append(pd.Series([round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant Topic', 'Percent Contribution', 'Topic Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

# # id tag words & frequency via corpus
id2word = corpora.Dictionary(dict_tokenizedAppReviews['com.abcdjdj.rootverifier']) #corpora 
corpus = [id2word.doc2bow(text) for text in dict_tokenizedAppReviews['com.abcdjdj.rootverifier']] #corpus based on assigned app

# ASSIGNED APP's lda model
LDAMODEL = dict_containLDAModels['com.abcdjdj.rootverifier']

# Take only review data from ASSIGNED APP
data = app5_df.loc[app5_df['Package name'] == 'com.abcdjdj.rootverifier'].Review

# fetch; map dominant topics
df_topic_sents_keywords = get_dominantTopics(ldamodel=LDAMODEL, corpus=corpus, texts=data)

df_dominant_topic.head(250) #prints first 250 reviews, in our case we only had 108 reviews, so all is printed




#### TASK 5: RECCOMENDATION
The apps that have been assigned to you are open source and their code are hosted on
GitHub. <br>
I. Look into the F-Droid page, GitHub Repository, and if available the issue tracking
system, Wikis, and website of your chosen ASSIGNED APP (from TASK 4). <br>
II. Recommend two new features or enhancements (could be bug reports) for the app
based on your above analysis that have not been done in the project. <br>
As the result you should have two suggestions for app improvement followed by the
discussion on how and why you came up with these recommendations <br><br>

Github Page: https://github.com/abcdjdj/RootVerifier-APP <br>
App Package name: `com.abcdjdj.rootverifier` <br><br>

First feature: <br>
A new feature that should be added is a way to have an in app system where users can add suggestions or bugs (i.e ticket system). From the 10 reviews I listed above (bonus task 4) we clearly see that nearly half of the reviews are bug reports and suggestions. By having a simple form system can provide a mutual benefits to both the users and the developers. Rather than having a flood of 1 star reviews for bug reports, users can easily send reviews through in-app and developer's app page will not get a lower rating as well as all the bug reports will be in a single spot. This allows developers to spend less time to manually go through the reviews to find bug reports. Likewise with suggestions, all new user suggestions can easily be organized through a form system and developers can easily prioritize which suggestions to implement with (assuming its under the scope of the app).<br>

e.g `suggestion found through review: great ui improve app l suggest make tweak calculation full function handy app dout`<br>
e.g `bug found through review: downloaded busy box nook tablet stay welcome screen busy box didnt root privelages pop dont know root authority im sure great apk cant seem figure<br>br>
As such it would be better to have in-app way of sending feedback to the developers, and reviews on the app store will look more appleasing to new users who are debating to try out the app incomparison to other competitors.`
<br>
Second feature: <br>
Since the main objective of this app is to verify if the current device is rooted or not. Some users have mistakenly downloaded the app to root the device. I can't simpily reccomend the developers to make it so the app can root the device as in google play's terms it's not allowed and the app needs kernal access I believe. So instead they should have a section in which it educates the users on how rooting a device works and explicitly stating device details on how the current device is not rooted. As such it would benefit users who leaves reviews like: <br>
`want something root tab please actually never rooted device one star isnt app thanks`

<br>
Both features would increase user satisfaction and leave less reviews in the app store and allow more users to try out the app instead of its competitors, also on a side note as mentioned earlier they should try to make seperate product line in which there's a lite version of the app that takes up less space and ONLY verifies if the device is rooted or not, and use this app to provide extra details.

#### BONUS of TASK 5 (3 Points): 
Open a new issue in the issue tracking system and suggest
the two recommend changes to the developers. Describe your logic and the data you
analyzed. List two URLS to the reported suggestions in your Notebook under the section
titled “Bonus of Task 5”.

PLEASE NOTE: I've put 2 suggestions under this 1 issue request, thank you
LINK: https://github.com/abcdjdj/RootVerifier-APP/issues/14 